File to explore IMS data from the paper: Predicting differential ion mobility behaviour in silico using machine learning.

Authors: Christian Ieritano, J. Larry Campbella, and W. Scott Hopkins

In [2]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit import RDLogger
# Disable RDKit error logging - prevent error messages from printing
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

import requests

from fcd_torch import FCD
import torch

In [19]:
file_path = '../data/ion_mobility_paper/predicting_ion_mobility_iertiano_et_al_data.csv'
data = pd.read_csv(file_path)

file_path = '../data/ion_mobility_paper/predicting_ion_mobility_iertiano_et_al_inchikeys.csv'
inchikey_df = pd.read_csv(file_path)

# Importing the MoNA database of mass spec data, just for use as database to go from compound names to smiles and inchikeys. May be irrelevant, now using PubChem's API to do this.
file_path = '../data/MoNA_data/MoNA-export-GC-MS_Spectra.sdf'
suppl = Chem.SDMolSupplier(file_path)

In [20]:
data.head()

,Compound,m/z,SV 1500,SV 2000,SV 2500,SV 3000,SV 3250,SV 3500,SV 3750,SV 4000,Boltzmann-weighted CCS
0,5-Cl-MQOH,194.04,0.1,0.1,0.3,0.5,0.91,1.6,2.57,3.8,138.66
1,5-CN-MQOH,185.07,0.1,0.1,-0.2,-0.3,-0.14,0.2,0.77,1.6,132.21
2,5-F-MQOH,178.07,-0.2,-0.4,-0.5,-0.3,0.08,0.7,1.60,2.8,127.47
3,5-Me-MQOH,174.09,-0.2,-0.2,0.0,0.8,1.47,2.4,3.65,5.3,129.45
4,5-NO2-MQOH,205.06,0.2,-0.1,0.0,-0.4,-0.34,0.0,0.59,1.4,137.66


In [21]:
compounds = list(data['Compound'])
print(f'There are {len(set(compounds))} compounds in the ion mobility dataset.')

There are 409 compounds in the ion mobility dataset.


In [22]:
print(f"There are {len(data['Compound'])} compounds in the ion mobility dataset and {len(inchikey_df['Compound'])} compounds in the corresponding inchikey file.")
print(f"Of those, {len(set(inchikey_df['Compound'])& set(data['Compound']))} overlap between the two files.")

There are 409 compounds in the ion mobility dataset and 409 compounds in the corresponding inchikey file.
Of those, 406 overlap between the two files.


In [23]:
print(set(inchikey_df['Compound']) - set(data['Compound']))
print(set(data['Compound']) - set(inchikey_df['Compound']))

{'Noroxycodone', 'Norsertraline', '(+)-Propoxyphene'}
{'Norsertraline HCL', 'Propoxyphene', 'Noroxycodone HCL'}


In [24]:
# changing the compound names in the inchikey df so they correspond to the names in the ims df
inchikey_df.loc[inchikey_df['Compound'] == '(+)-Propoxyphene'] = 'Propoxyphene'
inchikey_df.loc[inchikey_df['Compound'] == 'Norsertraline'] = 'Norsertraline HCL'
inchikey_df.loc[inchikey_df['Compound'] == 'Noroxycodone'] = 'Noroxycodone HCL'
print(set(data['Compound']) - set(inchikey_df['Compound']))

set()


In [ ]:
# Create list of comounds' corresponding inchikeys to use as column in dataframe
inchikeys = []

for row in data.iterrows():
    compound = row[1][0]
    inchikey = inchikey_df.loc[inchikey_df['Compound'] == compound, 'InChI Key'].values[0]
    inchikeys.append(inchikey)

print(f'Of the {len(inchikeys)} compounds in the dataset, {round(100 * (1 - pd.Series(inchikeys).isna().sum()/len(inchikeys)), 2)}% have corresponding InChIKeys.')

In [26]:
# add column for inchikeys to ims dataset
data_w_inchikey = data.copy()
data_w_inchikey['InChIKey'] = inchikeys

# double check that the InChIKeys column corresponds to correct compounds
total_errors = 0
for row in data_w_inchikey.iterrows():
    inchikey = row[1][-1]
    compound = row[1][0]
    try:
        correct_inchikey = inchikey_df.loc[inchikey_df['Compound'] == compound, 'InChI Key'].values[0]

    except:
        # for rows with no recorded inchikey, value = None
        correct_inchikey = None

    # if the inchikeys don't match and at least one of them is not none
    if inchikey != correct_inchikey:
        if inchikey == True or correct_inchikey == True:
            total_errors+=1

if not total_errors:
    print('All compound/InChIKey pairs are correct.')

All compound/InChIKey pairs are correct.


# Recording SMILES for each compound:
---
Chemception needs SMILES to generate embeddings. The ion mobility dataset has InChIKeys but no SMILES. Using code from [this](https://bioinformatics.stackexchange.com/questions/10755/is-there-a-python-package-to-convert-inchi-to-molecular-structures) question on Stack Exchange to get SMILES from [PubChem's API](https://pubchem.ncbi.nlm.nih.gov/docs/pug-rest) using InChIKeys.

In [32]:
# # get the corresponding SMILES for each InChIKey in the dataset and store in a list
# smiles_list = []
# for inchikey in data_w_inchikey['InChIKey']:
#     try:
#         # send inchikey to PubChem's API and get SMILES out of response
#         r = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{inchikey}/property/CanonicalSMILES/JSON').json()
#         smiles = r['PropertyTable']['Properties'][0]['CanonicalSMILES']
#         smiles_list.append(smiles)
#     except:
#         smiles_list.append(None)

In [94]:
# get the corresponding SMILES for each InChIKey in the dataset and store in a list
smiles_list = []
for inchikey in data_w_inchikey['InChIKey']:
    try:
        cid = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{inchikey}/cids/JSON')
        print(cid.json())
        # print(r.json())
        # data = r.json()
        # for dict_ in data['Hierarchies']['Hierarchy']:
        #     print(dict_['Information'].keys())
            # print(dict_.keys())
        # print(data['Hierarchies']['Hierarchy'][0].keys())
        # print(data['PC_Compounds'][0]['id'])
        break
    except:
        print('hi')
        break

{'IdentifierList': {'CID': [12945195]}}


In [119]:
# r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/12945195/classification/JSON')
# print(r.json())
thing = r.json()
# print(thing['Hierarchies']['Hierarchy'][0]['Information'])
hierarchies = thing.get('Hierarchies', {}).get('Hierarchy', [])
# hierarchies

In [53]:
def check_keys(data, print_keys = False, return_key_list=False):
    """
    Recursively prints all keys in a nested dictionary or list structure.

    Parameters:
    data (dict or list): The input data structure which can be a dictionary, list, 
                         or a combination of both. The function traverses the structure 
                         and prints each key it encounters.
    return_key_list (bool): If True, the function returns a list of all keys found
                             instead of just printing them.

    Returns:
    list or None: A list of keys if return_key_list is True; otherwise, None.
    """
    all_keys = []  # Initialize a list to store keys

    if isinstance(data, dict):  # Check if the input is a dictionary
        for key in data.keys():  # Iterate over all keys in the dictionary
            if print_keys:
                print(key)  # Print the current key
            all_keys.append(key)  # Add the key to the list
            # Recursively call check_keys on the value associated with the current key
            nested_keys = check_keys(data[key], print_keys, return_key_list)  # Get keys from nested dictionaries
            if nested_keys:  # If there are nested keys, add them to the main list
                all_keys.extend(nested_keys)

    elif isinstance(data, list):  # Check if the input is a list
        for item in data:  # Iterate through each item in the list
            nested_keys = check_keys(item, print_keys, return_key_list)  # Recursively check each item
            if nested_keys:  # If there are keys from the list items, add them to the main list
                all_keys.extend(nested_keys)

    if return_key_list:  # If the caller wants the list of keys
        return all_keys  # Return the accumulated list of keys


In [56]:
# r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/12945195/classification/JSON')
# response = r.json()
all_keys = check_keys(response, return_key_list=True)

In [12]:
# all_comps = len(data_w_inchikey['InChIKey'])
# comp_no_inchikey = data_w_inchikey['InChIKey'].isna().sum()
# comp_w_inchikey = all_comps - comp_no_inchikey
# comp_w_smiles = pd.Series(smiles_list).isna().sum()
# print(f"Of the {comp_w_inchikey} compounds with InChIKeys in the dataset, {round(100 * (1 - (comp_w_smiles-data_w_inchikey['InChIKey'].isna().sum())/comp_w_inchikey), 2)}% have corresponding SMILES on PubChem.")

Of the 399 compounds with InChIKeys in the dataset, 92.23% have corresponding SMILES on PubChem.


In [14]:
# data_w_inchikey['SMILES'] = smiles_list
# # remove rows without SMILES
# data_w_inchikey.dropna(subset=['SMILES'], inplace=True)
# data_w_inchikey.head()

,Compound,m/z,SV 1500,SV 2000,SV 2500,SV 3000,SV 3250,SV 3500,SV 3750,SV 4000,Boltzmann-weighted CCS,InChIKey,SMILES
0,5-Cl-MQOH,194.04,0.1,0.1,0.3,0.5,0.91,1.6,2.57,3.8,138.66,OPQODOXIDNYMKA-UHFFFAOYSA-N,CC1=NC2=C(C=CC(=C2C=C1)Cl)O
1,5-CN-MQOH,185.07,0.1,0.1,-0.2,-0.3,-0.14,0.2,0.77,1.6,132.21,NWUAFVKGPOBCOA-UHFFFAOYSA-N,CC1=NC2=C(C=CC(=C2C=C1)C#N)O
2,5-F-MQOH,178.07,-0.2,-0.4,-0.5,-0.3,0.08,0.7,1.60,2.8,127.47,YMDMCOFPYPOKJD-UHFFFAOYSA-N,CC1=NC2=C(C=CC(=C2C=C1)F)O
3,5-Me-MQOH,174.09,-0.2,-0.2,0.0,0.8,1.47,2.4,3.65,5.3,129.45,GQUFSGXAEOXQJC-UHFFFAOYSA-N,CC1=C2C=CC(=NC2=C(C=C1)O)C
4,5-NO2-MQOH,205.06,0.2,-0.1,0.0,-0.4,-0.34,0.0,0.59,1.4,137.66,XYPACLZTPMHPLB-UHFFFAOYSA-N,CC1=NC2=C(C=CC(=C2C=C1)[N+](=O)[O-])O


In [16]:
# data_w_inchikey.to_csv('../data/ion_mobillity_paper/ion_mobility_paper_data_w_inchikey.csv', index=False)

# Retrieving Chemception embeddings based on SMILES:
---

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fcd = FCD(device, n_jobs=1)
print(fcd)

In [58]:
# Collect class and superclass information for compounds, add as cols in df, save to csv
classes = []
superclasses = []

for row in data_w_inchikey.iterrows():
    compound = row[1][0]
    compound_class, compound_superclass = inchikey_df.loc[inchikey_df['Compound'] == compound, ['Class', 'Superclass']].values[0]
    classes.append(compound_class)
    superclasses.append(compound_superclass)

data_w_inchikey = pd.read_csv('../data/ion_mobility_paper/ion_mobility_paper_data_w_inchikey.csv')

data_w_inchikey_and_class = data_w_inchikey.copy()
data_w_inchikey_and_class['Class'] = classes
data_w_inchikey_and_class['Superclass'] = superclasses

data_w_inchikey_and_class.to_csv('../data/ion_mobility_paper/ion_mobility_paper_data_w_inchikeys_and_class_info.csv', index=False)

In [49]:
# get embedding and metadata for each compound and save to list 
embeddings = []

for smiles, inchikey, compound, in zip(data_w_inchikey['SMILES'], data_w_inchikey['InChIKey'], data_w_inchikey['Compound']):
    embedding = list(fcd.get_predictions([smiles])[0])
    embeddings.append(embedding + list([inchikey, compound, smiles]))


In [50]:
# Create column names for the embedding dataframe. First 512 col names are embedding_col_i, followed by inchikey, compound name and SMILES
col_names = []

for row in embeddings:
    for i, entry in enumerate(row):
        if type(entry) == np.float32:
            col_names.append('embedding_col_'+str(i))
    break

col_names.extend(['InChIKey', 'Compound Name', 'SMILES'])

In [53]:
embedding_df = pd.DataFrame(embeddings, columns=col_names)
embedding_df.head()

,embedding_col_0,embedding_col_1,embedding_col_2,embedding_col_3,embedding_col_4,embedding_col_5,embedding_col_6,embedding_col_7,embedding_col_8,embedding_col_9,...,embedding_col_507,embedding_col_508,embedding_col_509,embedding_col_510,embedding_col_511,InChIKey,Compound Name,SMILES,Class,Superclass
0,0.758398,0.004478,-0.425182,0.150473,-0.125434,0.008685,0.554332,-0.019404,-0.008864,-0.574413,...,-0.062587,-0.128178,0.546102,0.367006,0.252673,OPQODOXIDNYMKA-UHFFFAOYSA-N,5-Cl-MQOH,CC1=NC2=C(C=CC(=C2C=C1)Cl)O,Quinolines and derivatives,Organoheterocyclic compounds
1,0.634043,0.004474,-0.482186,0.103221,-0.109774,0.013885,0.605599,-0.049808,-0.019534,-0.682032,...,-0.118150,-0.510846,0.161031,0.421054,-0.046508,NWUAFVKGPOBCOA-UHFFFAOYSA-N,5-CN-MQOH,CC1=NC2=C(C=CC(=C2C=C1)C#N)O,Quinolines and derivatives,Organoheterocyclic compounds
2,0.731416,0.005359,-0.462983,0.180384,-0.101831,0.013309,0.567657,-0.098852,-0.011038,-0.640299,...,-0.071989,-0.212813,0.531222,0.359582,0.196023,YMDMCOFPYPOKJD-UHFFFAOYSA-N,5-F-MQOH,CC1=NC2=C(C=CC(=C2C=C1)F)O,Quinolines and derivatives,Organoheterocyclic compounds
3,0.640488,0.004132,-0.508613,-0.128938,-0.094348,0.010328,0.448675,-0.082013,-0.010061,-0.498070,...,-0.072403,-0.196029,0.587877,0.304906,0.174343,GQUFSGXAEOXQJC-UHFFFAOYSA-N,5-Me-MQOH,CC1=C2C=CC(=NC2=C(C=C1)O)C,Quinolines and derivatives,Organoheterocyclic compounds
4,0.721129,0.006346,-0.513118,0.030674,-0.276892,0.005546,0.721887,-0.040932,-0.013259,-0.406169,...,-0.068566,0.577365,0.557190,0.144175,0.425554,XYPACLZTPMHPLB-UHFFFAOYSA-N,5-NO2-MQOH,CC1=NC2=C(C=CC(=C2C=C1)[N+](=O)[O-])O,Quinolines and derivatives,Organoheterocyclic compounds


In [55]:
embedding_df.to_csv('../data/ion_mobility_paper/ion_mobility_paper_embeddings.csv', index=False)

# Using MoNA as InChIKey lookup:
---
**This was the previous approach before finding the PubChem API:**

Since the MoNA dataset has both SMILES and InChIKey, using that as a way to go from one to the other. I haven't found a way to do this more efficiently, the other option I'm aware of is manually looking up each compound's InCHIKey somewhere like NIST Webbook, which is VERY time consuming and prone to human error (namely me copying things incorrectly). 

MoNA samples contain SMILES in the comment, not as a property, so the SMILES must be pulled out of the comment to use.

In [12]:
def format_smiles(mol):
    """
    Extracts the SMILES string from the COMMENT property of an rdkit molecule.

    Parameters:
    mol: An rdkit molecule.

    Returns:
    str or None: The SMILES string if found, otherwise None.
    """
    
    comment = mol.GetProp('COMMENT').split('\n')
    smiles = None
    for line in comment:
        if line.split('=')[0] == 'SMILES':
            smiles = line.split('SMILES=')[1]

    return smiles

In [29]:
# get SMILES for any compounds in the MoNA database that are also in the Ion Mobiity dataset and create a dictionary with inchikeys as keys and SMILES as values
overlapping_mol_inchikey_smiles_dict = {}

for mol in suppl:
    if mol is not None:
        try:
            inchikey = mol.GetProp('INCHIKEY')

        # ignore any mols without inchikeys
        except:
            continue
        
        # if this compound is in the ion mobility dataset
        if inchikey in compound_inchikey_dict.values():
            # if this compound's smiles hasn't been recorded yet
            if not inchikey in overlapping_mol_inchikey_smiles_dict.keys():
                smiles = format_smiles(mol)
                if smiles:
                    overlapping_mol_inchikey_smiles_dict[inchikey] = smiles


In [30]:
# Get a list of compounds from the ion mobility dataset that also appear in the MoNA dataset.
compounds_in_both_datasets = []
for mol in suppl:
    if mol is not None:
        if mol.GetProp('NAME') in compounds:
            compounds_in_both_datasets.append(mol.GetProp('NAME'))

We see that there are more InChIKeys in common between the datasets than there are compounds names. This is likely due to the fact that InChIKeys are unique identifiers while the same chemical might can have multiple compound names, meaning that it could be listed one way in one dataset and another way in another.

In [31]:
print(f'There are {len(set(compounds_in_both_datasets))} compound names from the Ion Mobility dataset that appear in the MoNA dataset compared to {len(overlapping_mol_inchikey_smiles_dict)} inchikeys that appear in both datasets.')

There are 48 compound names from the Ion Mobility dataset that appear in the MoNA dataset compared to 76 inchikeys that appear in both datasets.


In [38]:
# make lists of inchikeys and SMILES to add as columns to the IMS dataframe
inchikeys = []
smiles = []

for mol in compounds:
    inchikey = compound_inchikey_dict[mol]
    
    # look mol up in dict and append corresponding inchikey to list
    inchikeys.append(inchikey)
    
    # if inchikey is in the dict of mols with SMILES from MoNA, get the SMILES and append to list
    if inchikey in overlapping_mol_inchikey_smiles_dict.keys():
        smiles.append(overlapping_mol_inchikey_smiles_dict[inchikey])
    else:
        smiles.append(None)

NameError: name 'compound_inchikey_dict' is not defined

In [33]:
# Double checking that the recorded SMILES/InChIKey pairs correspond
total_errors = 0
for i, smile in enumerate(smiles):
    if smile:
        if not overlapping_mol_inchikey_smiles_dict[inchikeys[i]] == smiles[i]:
            print(f'SMILES/InChIKey pair at {i} does not match what is specified in the lookup dictionary.')
            total_errors+=1
if not total_errors:
    print('All SMILES/InChIKey pairs match what is specified in the lookup dictionary.')

All SMILES/InChIKey pairs match what is specified in the lookup dictionary.


Adding columns for InChIKey and SMILES. InChIKey will be needed for identification, SMILES for retrieving Chemception embeddings.

In [34]:
ims_data = data.copy()
ims_data['SMILES'] = smiles
ims_data['InChIKey'] = inchikeys
ims_data = ims_data.dropna(subset=['SMILES']).reset_index(drop=True)
# ims_data.drop(columns=['Compound'], inplace=True)
ims_data.head()

,Compound,m/z,SV 1500,SV 2000,SV 2500,SV 3000,SV 3250,SV 3500,SV 3750,SV 4000,Boltzmann-weighted CCS,SMILES,InChIKey
0,6-methyl-2-MQ,158.10,0.20,0.20,0.60,1.90,3.03,4.50,6.32,8.50,126.14,Cc(c2)cc(c1)c(c2)nc(C)c1,JJPSZKIOGBRMHK-UHFFFAOYSA-N
1,MQOH,160.08,-0.40,-0.60,-0.70,-0.40,0.06,0.80,1.89,3.40,125.75,Cc(c1)nc(c2)c(ccc2)c1,SMUQFGGVLNAIOZ-UHFFFAOYSA-N
2,Ametryn,228.13,0.44,1.15,2.38,4.04,5.07,6.37,7.67,9.29,151.92,CCNc(n1)nc(SC)nc(NC(C)C)1,RQVYBGPQFYCBGX-UHFFFAOYSA-N
3,Azoxystrobin,404.13,0.66,1.65,3.14,5.46,7.03,8.77,10.78,13.03,203.24,CO/C=C(\C1=CC=CC=C1OC2=NC=NC(=C2)OC3=CC=CC=C3C...,WFDXOXNFNRHQEC-GHRIWEEISA-N
4,Dimethoate,230.01,0.48,1.32,2.71,4.80,6.18,7.88,9.57,11.49,140.98,CNC(=O)CSP(=S)(OC)OC,MCWXGJITAZMZEV-UHFFFAOYSA-N
